In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install lofo-importance

<hr>

**LOFO_Importance:****

1. It will remove one feature from the list of features and calculate the performance of the base_model that we have given with the scoring parameter.

> **why lofo:**
1. There are a lot of methods to select important features like forward feature selection, Tree-based feature selection, and Recursive Feature Elimination
2. LOFO has several advantages compared to other:
3. It is model agnostic. You won’t need different importance types using different models. As we use weights for Logistic Regression and gain for GBDT
4. It gives negative importance to features that hurt performance upon inclusion

In [ ]:
import pandas as pd
import numpy as np
import janestreet as jane
from lofo import LOFOImportance, Dataset, plot_importance
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train = pd.read_csv("/kaggle/input/jane-street-market-prediction/train.csv")

In [ ]:

train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)
features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
X_train = list(list(np.where(train.columns.str.contains("feature")))[0]) 
X_train = list(train.columns[X_train])

In [ ]:
train['Y'] = ((train['resp'].values) > 0).astype(int)

In [ ]:
train.head(20)

In [ ]:
Y_train = list(list(np.where(train.columns.str.contains("Y")))[0]) 
Y_train = list(train.columns[Y_train])

In [ ]:
dataset = Dataset(df=train, target=Y_train, features=X_train)

In [ ]:
Imp = LOFOImportance(dataset, cv=3, scoring="roc_auc")

In [ ]:
imp_result = Imp.get_importance()

In [ ]:
imp_result.to_csv("importanceResults_resp.csv", index=False)

In [ ]:

plot_importance(imp_result.head(130), figsize=(8, 70), kind="default")

**Missing Value Analysis**

In [ ]:
temp = pd.DataFrame(train.isna().sum().sort_values(ascending=False)*100/train.shape[0],columns=['missing %']).head(20)
temp.style.background_gradient(cmap='Purples')